# I. Lý thuyết

1) Transfer Learning thường được thực hiện trên hai dữ liệu nguồn (huấn luyện mô hình nguồn) và dữ liệu đích (huấn luyện mô hình đích) thế nào ?


A. Dữ liệu nguồn và dữ liệu đích có sự liên quan tới nhau. Những đặc trưng trong dữ liệu đích xuất hiện ở những dữ liệu nguồn.

B. Dữ liệu nguồn có số lượng classes lớn hơn dữ liệu đích.

C. Kích thước của dữ liệu nguồn rất nhỏ.

D. Dữ liệu đích ít liên quan tới dữ liệu nguồn.

=> Đáp án A

2) Khi nào thì chúng ta nên thực hiện fine tuning trên toàn bộ các layers của mô hình đích ?

A. Khi hai dữ liệu nguồn và đích có mức độ tương đồng cao và dữ liệu đích có kích thước lớn.

B. Khi hai dữ liệu nguồn và đích có mức độ tương đồng cao và dữ liệu đích có kích thước nhỏ.

C. Dữ liệu đích có kích thước nhỏ và dữ liệu mục tiêu có kích thước lớn.

D. Cả hai dữ liệu đích và mục tiêu đều có kích thước nhỏ.

=> Đáp án A

3) Các phương pháp augmentation nào sẽ kết hợp nội dung từ hai ảnh lẫn nhau và tạo ra một nhãn mềm (_soft label_) cho ảnh?

A. Rotation, Random Crop, MixUp

B. Bright Constrast, Color Shift, Addition Noise

C. CutMix, MixUp

D. Flip, Information Loss 

=> Đáp án C

4) Quá trình xây dựng một mô hình AI trong dự án là một chu trình Machine Learning Cycle kế hợp giữa huấn luyện và gán nhãn dữ liệu. Để tiết kiệm chi phí gán nhãn chúng ta nên sử dụng phương pháp nào ?

A. Lấy mẫu ngẫu nhiên từ tập unlabeled dataset để thực hiện gán nhãn.

B. Sử dụng Active Learning để lựa chọn mẫu mang lại thông tin giúp cải thiện nhiều nhất cho hiệu suất mô hình.

C. Lựa chọn mô hình pretrained lớn nhất có thể.

D. Chỉ lựa các dữ liệu có thông tin rõ ràng, có thể phân biệt được bởi con người.

=> Đáp án B

5) Mô hình lớn thường đạt độ chính xác cao nhưng không deploy được trên các thiết bị edge device, IoT,... Trong khi mô hình nhỏ có thể deploy được nhưng thường có độ chính xác thấp. Phương pháp nào có thể giúp mô hình nhỏ cải thiện được độ chính xác ? Có thể lựa chọn nhiều đáp án.

A. Sử dụng active learning để lựa chọn các mẫu đại diện cho tổng thể để huấn luyện mô hình nhỏ.

B. Áp dụng augmentation để huấn luyện mô hình nhỏ.

C. Fine tuning các layers của mô hình lớn sang mô hình nhỏ.

D. Sử dụng knowledge distillation để chuyển giao tri thức từ mô hình lớn sang mô hình nhỏ.

=> Đáp án B, C, D

# II. Thực hành

6) Từ bộ dữ liệu [Dog and Cat](https://www.kaggle.com/c/dog-vs-cat-classification/data), hãy huấn luyện một mô hình large (chẳng hạn ResNet50) bằng cách fine-tuning lại các trọng số từ pretrained model của bộ dữ liệu ImageNet. Huấn luyện trên 5 epochs.

7) Hãy huấn luyện một mô hình small (chẳng hạn MobileNetV3) không sử dụng pretrained model trên 1 epochs.

8) Sử dụng mô hình large làm teacher để cải thiện mô hình small là student theo phương pháp knowledge distillation.

9) Áp dụng thêm các kĩ thuật data augmentation kết hợp ảnh khác nhãn để tạo thành nhãn mềm và huấn luyện cải thiện tiếp model student.

10) Giả định cần huấn luyện tiếp mô hình student với các dữ liệu mới chưa được gán nhãn. Hãy xây dựng một kĩ thuật lựa chọn mẫu dựa trên đánh giá uncertainty.

## 6.Huấn luyện large model Resnet50 sử dụng fine-tuning trên 5 epochs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import glob
train_dir = '/content/drive/MyDrive/train/train'
test_dir = '/content/drive/MyDrive/test/test'
train_images = glob.glob(train_dir + "/**/**.jpg")
test_images = glob.glob(test_dir + "/**.jpg")

In [ ]:
train_images[:5]

['/content/drive/MyDrive/train/train/cats/cat.9072.jpg',
 '/content/drive/MyDrive/train/train/cats/cat.9356.jpg',
 '/content/drive/MyDrive/train/train/cats/cat.916.jpg',
 '/content/drive/MyDrive/train/train/cats/cat.9384.jpg',
 '/content/drive/MyDrive/train/train/cats/cat.8956.jpg']

In [ ]:
dogs_list = [img for img in train_images if img.split("/")[-2] == "dogs"]
cats_list = [img for img in train_images if img.split("/")[-2] == "cats"]

print("The number of dogs images: ",len(dogs_list))
print("The number of cats images: ",len(cats_list))

class_to_int = {"dogs" : 0, "cats" : 1}
int_to_class = {0 : "dogs", 1 : "cats"}

The number of dogs images:  12500
The number of cats images:  12500


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (128, 128)
batch_size = 32

print("Getting Data...")
datagen = ImageDataGenerator(rescale=1./255, # normalize pixel values
                             validation_split=0.3, # hold back 30% of the images for validation
                             rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,
                             zoom_range=0.2,shear_range=0.2)

print("Preparing training dataset...")
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classnames = list(train_generator.class_indices.keys())
print('Data generators ready')

Getting Data...
Preparing training dataset...
Found 17500 images belonging to 2 classes.
Preparing validation dataset...
Found 7500 images belonging to 2 classes.
Data generators ready


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

resnet_model = Sequential()
pretrained_model = tf.keras.applications.ResNet50(input_shape=(128,128,3),
                                include_top=False,pooling='avg',
                                classes=2,weights='imagenet')
# Đóng băng các lớp layers để ko train nữa
for layer in pretrained_model.layers:
    layer.trainable=False
# => Trainable params = 0
resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512,'relu'))
resnet_model.add(Dropout(0.4))
resnet_model.add(Dense(2,'softmax'))

In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 24,637,826
Trainable params: 1,050,114
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
 resnet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
epochs = 5 
history_resnet = resnet_model.fit(
                        train_generator,
                        steps_per_epoch = train_generator.samples // batch_size,
                        validation_data = validation_generator, 
                        validation_steps = validation_generator.samples // batch_size,
                        epochs = epochs)

Epoch 1/5
546/546 [==============================] - 494s 906ms/step - loss: 0.6794 - acc: 0.5678 - val_loss: 0.6834 - val_acc: 0.5437
Epoch 2/5
546/546 [==============================] - 454s 832ms/step - loss: 0.6729 - acc: 0.5830 - val_loss: 0.6731 - val_acc: 0.5838
Epoch 3/5
546/546 [==============================] - 395s 724ms/step - loss: 0.6690 - acc: 0.5920 - val_loss: 0.6554 - val_acc: 0.6237
Epoch 4/5
546/546 [==============================] - 346s 633ms/step - loss: 0.6665 - acc: 0.5943 - val_loss: 0.6737 - val_acc: 0.5819
Epoch 5/5
546/546 [==============================] - 321s 588ms/step - loss: 0.6622 - acc: 0.6042 - val_loss: 0.6690 - val_acc: 0.5813


## 7. Huấn luyện small model MobileNetV3 không sử dụng pretrained model trên 1 epoch

In [ ]:
with strategy.scope():
  mobilenet_model = Sequential()
  pretrained_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3),
                                include_top=False,pooling='avg',
                                weights='imagenet')
  # Đóng băng các lớp layers để ko train nữa
  for layer in pretrained_model.layers:
    layer.trainable=False
  # => Trainable params = 0
  mobilenet_model.add(Flatten())
  mobilenet_model.add(Dense(2,'softmax'))
  mobilenet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [ ]:
epochs = 1
history_mobilenet_model = mobilenet_model.fit(
                          train_generator,
                          steps_per_epoch = train_generator.samples // batch_size,
                          validation_data = validation_generator, 
                          validation_steps = validation_generator.samples // batch_size,
                          epochs = epochs)

546/546 [==============================] - 670s 1s/step - loss: 1.6959 - acc: 0.5268 - val_loss: 1.3208 - val_acc: 0.5212


## 8.Sử dụng mô hình large làm teacher để cải thiện mô hình small là student theo phương pháp knowledge distillation.